# Taller Playfair

**Estudiante:** Omar David Toledo Leguizamón

Implementar el algoritmo de cifrado de bloques Playfair Cipher en Python. Utilizar un parámetro para identificar si es para cifrar o descifrar, por ejemplo, 1 o 0.

1. El algoritmo debe recibir como entrada para cifrar:
* El mensaje en texto claro
* La clave
2. Para decifrar:
* Mensaje cifrado
* La clave

## Desarrollo

Primero definimos una función que dada una frase llave de entrada, construya la matriz usada para cifrar. Adicionalmente, construiremos un diccionario que permita realizar acceso rapido de cada letra a su posicion asociada. (Con esto, hacemos que la identificación de la posición se haga en tiempo $O(1)$ y no en tiempo $O(n^2)$, siendo $n$ la dimensión de la matriz)

In [1]:
import numpy as np

def buildMatrix(key = ""):
    key = key.upper()
    m = np.zeros(shape = (5,5), dtype = '<U2')
    fastAccessDict = {}
    i = 0
    j = 0
    for char in key:
        if char != ' ':
            c = 'IJ' if char == 'I' or char=='J' else char
            if c not in fastAccessDict:
                for l in c:  fastAccessDict[l] = (i,j)
                m[i,j] = c
                j = (j+1)%5
                if j==0: i=(i+1)%5
    for k in range(65, 65+25):
        c = chr(k)
        if c not in fastAccessDict:
            fastAccessDict[c] = (i,j)
            m[i,j] = c
            j = (j+1)%5
            if j==0: i=(i+1)%5

    return m, fastAccessDict


Podemos ver como funciona el generador de la matriz con la clave *Yoan Pinzon*

In [2]:
m,d = buildMatrix("Yoan Pinzon")
print(m)

[['Y' 'O' 'A' 'N' 'P']
 ['IJ' 'Z' 'B' 'C' 'D']
 ['E' 'F' 'G' 'H' 'K']
 ['L' 'M' 'Q' 'R' 'S']
 ['T' 'U' 'V' 'W' 'X']]


Ahora hacemos una función que dada una entrada, la separa en pares de letras válidas para ser cargadas en el algoritmo

In [3]:
def splitEntry(text = ""):
    text = text.upper()
    text = ''.join(text.split(' '))
    pairs = []
    i = 0
    while(i < len(text)-1):
        if text[i]!=text[i+1]:
            pairs.append(text[i]+text[i+1])
            i+=2
        else:
            pairs.append(text[i]+'X')
            i+=1
    if i==len(text)-1: pairs.append(text[i]+'X')
    return pairs

La funcion agrega padding donde es necesario (Cuando hay dos letras consecutivas y pueden caer en pareja; y cuando se tienen una letra aislada al final del texto)

In [4]:
' '.join(splitEntry(text = "This secret message is encrypted"))

'TH IS SE CR ET ME SX SA GE IS EN CR YP TE DX'

Ahora realizamos la funcion que aplica el cifrado Playfair dada una cadena de entrada y una llave

In [5]:
def playfairCipher(text = '', key = ''):
    m, d = buildMatrix(key = key)
    pairs = splitEntry(text = text)
    codedPairs = []
    for pair in pairs:
        pos1 = d[pair[0]]
        pos2 = d[pair[1]]
        if pos1[1] == pos2[1]:
            l1 = m[(pos1[0]+1)%5,pos1[1]]
            l2 = m[(pos2[0]+1)%5,pos2[1]]
        elif pos1[0] == pos2[0]:
            l1 = m[pos1[0],(pos1[1]+1)%5]
            l2 = m[pos2[0],(pos2[1]+1)%5]
        else:
            l1 = m[pos1[0],pos2[1]]
            l2 = m[pos2[0],pos1[1]]
        codedPairs.append(l1+l2)
    return codedPairs


In [6]:
' '.join(playfairCipher(text = 'This secret message is encrypted', key = 'Yoan Pinzon'))

'WE DL LK HW LY LF XP QP HF DL HY HW OY YL KP'

Ahora hacemos la rutina encargada de decifrar una entrada dada una clave

In [7]:
def playfairDecipher(text = '', key = ''):
    m, d = buildMatrix(key = key)
    pairs = splitEntry(text = text)
    codedPairs = []
    for pair in pairs:
        pos1 = d[pair[0]]
        pos2 = d[pair[1]]
        if pos1[1] == pos2[1]:
            l1 = m[(pos1[0]-1)%5,pos1[1]]
            l2 = m[(pos2[0]-1)%5,pos2[1]]
        elif pos1[0] == pos2[0]:
            l1 = m[pos1[0],(pos1[1]-1)%5]
            l2 = m[pos2[0],(pos2[1]-1)%5]
        else:
            l1 = m[pos1[0],pos2[1]]
            l2 = m[pos2[0],pos1[1]]
        codedPairs.append(l1+l2)
    return codedPairs


In [8]:
' '.join(playfairDecipher(text = 'WE DL LK HW LY LF XP QP HF DL HY HW OY YL KP',
                         key = 'Yoan Pinzon'))

'TH IJS SE CR ET ME SX SA GE IJS EN CR YP TE DX'

Finalmente diseñamos una función que maneje las entradas y salidas del problema

In [13]:
def PlayfairMain():
    print('Inicio de programa de cifrado-descifrado usando Playfair\n')
    mode = int(input('Elija el modo de operación (1 para cifrar, 2 para descifrar, 3 para salir): '))
    if mode==1:
        text = str(input('Ingresa el texto a cifrar: '))
        key = str(input('Ingresa la llave para cifrar: '))
        answer = playfairCipher(text,key)
        answer = ' '.join(answer)
        print(f'Texto cifrado: {answer}')
    elif mode==2:
        text = str(input('Ingresa el texto a descifrar: '))
        key = str(input('Ingresa la llave para descifrar: '))
        answer = playfairDecipher(text,key)
        answer = ' '.join(answer)
        print(f'Texto descifrado: {answer}')
    else:
        return
    print()
    PlayfairMain()

In [14]:
PlayfairMain()

Inicio de programa de cifrado-descifrado usando Playfair

Elija el modo de operación (1 para cifrar, 2 para descifrar, 3 para salir): 1
Ingresa el texto a cifrar: This secret is encrypted
Ingresa la llave para cifrar: Yoan Pinzon
Texto cifrado: WE DL LK HW LY DL HY HW OY YL KP

Inicio de programa de cifrado-descifrado usando Playfair

Elija el modo de operación (1 para cifrar, 2 para descifrar, 3 para salir): 2
Ingresa el texto a descifrar: WE DL LK HW LY DL HY HW OY YL KP
Ingresa la llave para descifrar: Yoan Pinzon
Texto descifrado: TH IJS SE CR ET IJS EN CR YP TE DX

Inicio de programa de cifrado-descifrado usando Playfair

Elija el modo de operación (1 para cifrar, 2 para descifrar, 3 para salir): 3
